# Introduction to graphs and shortest paths problems

For this lab session, we will be using a widespread Python library for graphs called [`networkx`](https://networkx.github.io/).
Should you face any issue during this session, your main reference will be [this one](https://networkx.github.io/documentation/stable/).

The purpose of this session is not to evaluate you. All short exercices are provided with their solutions. However, next session will be graded so you are highly encouraged to try to solve the small problems by yourself before looking for answers.

Each time you encounter this kind of cell:
```python
# %load solution.py
```
you may uncomment the line and execute the cell to load the solution. Execute the cell again to run the solution.

Be careful if you write anything in the cell containing the `%load` command: your code will be deleted when you try to reveal the solution. Use a different cell for your attempts.

## Introduction to networkx

<div class="alert alert-warning">
    <b>Exercice:</b> Import the <code>networkx</code> library. Use the common <code>nx</code> abbreviation.
</div>

In [ ]:
# Write your code here


In [ ]:
# %load solutions/import_networkx.py


There are many ways to feed nodes and edges to a graph.
Nodes may be of any type as long as it is [hashable](https://docs.python.org/3/glossary.html) (lists are not allowed since they can be modified by side effects).

In [ ]:
g = nx.Graph()
g.add_node("spam")
# a node is created if it does not exist when you create an edge
g.add_edge(1, 2)

g.nodes, g.edges

There are several ways to display graphs. You may explore different possibilities in the documentation. We will mostly use the matplotlib output in this notebook. The layout of the graph is done automatically, but you can select another one with the `pos` parameter of `draw_networkx`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ax = plt.axes()
ax.set_axis_off()
ax.set_aspect(1)
nx.draw_networkx(g, ax=ax)

### Directed graphs, weighted graphs

You may create undirected (`Graph`) or directed (`DiGraph`) graphs. For convenience sake, you may feed an adjacency list directly when constructing the graph.

For a weighted graph, just add the corresponding parameter to the edge. The bracket notation returns an edge as a dictionary with all the parameters given at creation.

In [ ]:
g = nx.DiGraph()

# create edges from an iterable
g.add_edges_from(
    zip([0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5]),
    # you may add weights to edges just naturally
    weight=5,
)

ax = plt.axes()
ax.set_axis_off()
ax.set_aspect(1)
nx.draw_networkx(g, ax=ax)

g[0][1]

<div class="alert alert-warning">
    <b>Exercice:</b> Create the following graphs with <code>networkx</code>.
</div>

![graphs](img/graphs.png)

In [ ]:
# Write your code here


In [ ]:
# %load solutions/create_graph.py


Graphs are provided with methods and properties that were defined in class.

In [ ]:
# the adjacency list for the graph
g.adj

In [ ]:
# the edges leaving from A
g['A']

In [ ]:
# the degree of node A
g.degree['A']

In [ ]:
g.is_directed()

In [ ]:
# you may create the undirected version of a directed graph
g_undirected = g.to_undirected()
# and compare the adjacency lists
g_undirected.adj

## Shortest paths problems

The `networkx` library contains many graph algorithms implementations.

In [ ]:
import pkgutil
list(module for _, module, is_package in pkgutil.iter_modules(nx.algorithms.__path__)
     if is_package)

### Basic introduction to Dijkstra algorithm

We will focus in this session on shortest paths problems. Let us consider the following graph we studied in class.

In [ ]:
g = nx.DiGraph()

g.add_edge('A', 'B', weight=4)
g.add_edge('A', 'C', weight=2)
g.add_edge('B', 'C', weight=3)
g.add_edge('B', 'D', weight=2)
g.add_edge('B', 'E', weight=3)
g.add_edge('C', 'D', weight=4)
g.add_edge('C', 'E', weight=5)
g.add_edge('E', 'D', weight=1)

ax = plt.axes()

nx.draw_networkx(
    g, ax=ax,
    pos=nx.shell_layout(g)
)
nx.draw_networkx_edge_labels(
    g, ax=ax, pos=nx.shell_layout(g),
    edge_labels=dict(((u, v), g[u][v]['weight'])
                     for u, v in g.edges)
)

ax.set_aspect(1)
ax.set_axis_off()


<div class="alert alert-hint">
    <b>Dijkstra algorithm</b>: Time to recall the principle of the algorithm!
</div>

Dijkstra algorithm is provided in `networkx`; a more advanced interface is also available if we want to unroll the algorithm.

In [ ]:
from matplotlib import animation

def anim_to_html(anim):
    plt.close(anim._fig)
    return anim.to_html5_video()

animation.Animation._repr_html_ = anim_to_html

In [ ]:
fig, ax = plt.subplots()
ax.set_axis_off()
ax.set_aspect(1)

nx.draw_networkx(
    g, ax=ax,
    pos=nx.shell_layout(g)
)

def animate(i):

    predecessors, _ = nx.shortest_paths.dijkstra_predecessor_and_distance(g, 'A', i)
    nx.draw_networkx_edges(
        g, ax=ax, pos=nx.shell_layout(g),
        edgelist=[(succ, pred)
                  for pred in predecessors.keys()
                  for succ in predecessors[pred]],
        edge_color='crimson',
        width=2
    )
    nx.draw_networkx_edge_labels(
        g, ax=ax, pos=nx.shell_layout(g),
        edge_labels=dict(((u, v), g[u][v]['weight'])
                         for u, v in g.edges)
    )
    return []

animation.FuncAnimation(fig, animate, frames=15,
                        interval=750, blit=True)


### A study case on the subway path finding problem

In this study case, we will be using the [RATP Open Data](https://data.ratp.fr/explore/dataset/offre-transport-de-la-ratp-format-gtfs/) service. You will find in the `data/` directory the `RATP_GTFS_LINES.zip` archive which can be found on the website.

The [GTFS format](https://en.wikipedia.org/wiki/General_Transit_Feed_Specification) is a format to communicate about transport services. The following script parses the archive and creates four dictionaries:

 - `stations` contains information about the stations (id, name, lat/lon positions);
 - `stop_times` contains train schedules for each station (stop_id, trip_id, arrival_time);
 - `trips` contains a one-to-n relationship between trips and routes (one trip_id per day, several trip_id for one route_id);
 - `transfers` contains minimum transfer times (walking time) from one station to another (more precisely)

Please note:

 - all dictionaries are indexed by the name of the line (`METRO_13`, `RER_B`);
 - the same station **may have several stop_id and several lat/lon coordinates**. The stop_id is associated to a subway platform rather than to a station building;
 - bus lines have been ignored in order to keep a map readable but the archive contains all you need to build your own pathfinding app with the full network information (you may want to also include [this](https://ressources.data.sncf.com/explore/dataset/sncf-transilien-gtfs/information/) or [this](https://data.toulouse-metropole.fr/explore/dataset/tisseo-gtfs/table/) information);
 - we use a [Lambert 93](https://fr.wikipedia.org/wiki/Projection_conique_conforme_de_Lambert#Projections_officielles_en_France_m%C3%A9tropolitaine) projection to convert lat/lon to x-y coordinates;
 - **you may use euclidean distance on x-y coordinates**.

<div class="alert alert-danger">
    <b>Warning:</b> The following looks long but all the work has been done for you. The real work starts after the first map of the subway network.
</div>

In [ ]:
# This contains many useful functions you don't need to waste time on
# (but you may have a look if you are curious...)

%run shortest_paths.py

You may explore the four tables for metro line 1 as follows:

In [ ]:
from IPython.display import display, HTML

display(HTML("<h4>Stations on METRO_1</h4>"))
display(stations['METRO_1'].head())

display(HTML("<h4>Stop times on METRO_1</h4>"))
display(stop_times['METRO_1'].head())

display(HTML("<h4>Trips on METRO_1</h4>"))
display(trips['METRO_1'].head())

display(HTML("<h4>Transfer times on METRO_1</h4>"))
display(transfers['METRO_1'].head())

> Just for fun, before we get back to work, see how we can use the power of pandas data frames in order to get all the mission codes associated to RER B.
The first letter of the mission code is associated to the terminus station of the train
(*but this is useless for us now so **you may forget it** unless you are a nerd with transportation networks*)

In [ ]:
index = 'RER_B'

(trips[index]
 # keep only one trip_id
 .drop_duplicates('trip_headsign')
 # get all station_id associated to a trip_id
 .merge(stop_times[index])
 # get the station names associated to the station_id
 .merge(stations[index])
 # be sure all stops come in order
 .sort_values('stop_sequence')
 # for each mission code
 .groupby('trip_headsign')
 # only keep the last station name on the route.
 [['stop_name']].last()
 # Reverse the table and
 .reset_index()
 # for each last station
 .groupby('stop_name')
 # put all mission codes into one string
 .apply(lambda df: ", ".join(df.trip_headsign))
 # finally, export to a Python dictionary to make it more readable
 .to_dict())

#### Data exploration

In order to get a better grasp on the data, let's look at line `METRO_13`:

 - the two direction ids;
 - the four different route_id because of the two branches to the North;
 - all stations in order for one trip_id.

In [ ]:
trips['METRO_13'].drop_duplicates('trip_headsign')

In [ ]:
index = 'METRO_13'

(trips[index]
 # keep only one trip_id
 .head(1)
 # get all station_id associated to a trip_id
 .merge(stop_times[index])
 # get the station names associated to the station_id
 .merge(stations[index])
 # be sure all stops come in order
 .sort_values('stop_sequence')
 # filter information
 [['stop_id', 'stop_name', 'departure_time']]
)

In [ ]:
all_stations[['stop_name', 'line']].query("stop_name == 'Saint-Lazare'")

Since we don't want to play with `stop_id`, things can get complicated with capital letters, accentuated letters, hyphens and typos (don't get me started!), we provide a fuzzy search method.

`.` is a wildcard character, and you may need `$` to mark the end of the string.

Also, the variable `pos` contains all x-y coordinates for each station_id. Remember you may use euclidean distances.

In [ ]:
search_station('Saint.Laz')

In [ ]:
{ # Nation also matches "Nationale"
    key: value
    for key, value in search_station('Nation').items()
    if value != 'Nation'
}

In [ ]:
# Use the wildcard
search_station('Nation$')

In [ ]:
import numpy as np

i1, n1 = search_station('Nation$').popitem()
x1, y1 = pos[i1]
i2, n2 = search_station('Saint-Laz').popitem()
x2, y2 = pos[i2]

# in meters
d = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

print(f"Distance between {n1} and {n2}: {d/1000:.2f} km")

#### Basic graph of the network

Now, we got all we need to build a first basic graph of the network.

You will need to fill the graph with more information later so **take the time to understand what happens here**:

In [ ]:
g = nx.DiGraph()

# for each subway line
for line_nb, trip in trips.items():

    # for each different trip
    for trip_id in trip.drop_duplicates('route_id').trip_id:

        # Note:
        #   - we should use "trip_headsign" rather than "route_id"
        #     but the map gets confusing outside Paris
        #   - inside Paris, the graph we build here is still correct

        sequence = list(
            # get the list of stop_id in orders
            stop_times[line_nb]
            .query(f'trip_id == {trip_id}')
            .sort_values('stop_sequence')
            .stop_id
        )

        for first, second in zip(sequence[:-1], sequence[1:]):
            g.add_edge(
                first, second,
                # we store 'RER' or 'METRO' for printing it differently
                type=line_nb.split('_')[0],
                # line_colors is provided as is
                color=line_colors[line_nb]
            )

# for each subway line
for line_nb, transfer in transfers.items():

    # parse the lines of the table in order
    for _, line in transfer.iterrows():

        first, second = line.from_stop_id, line.to_stop_id

        # add an edge for each connection if both nodes already exists in the graph
        # (remember there are a lot of bus stations we chose to ignore)
        if first in g.nodes and second in g.nodes:
            g.add_edge(
                first, second,
                type='CONNECTION',
                duration=line.min_transfer_time,
                color='#aaaaaa'
            )
            g.add_edge(
                second, first,
                type='CONNECTION',
                duration=line.min_transfer_time,
                color='#aaaaaa'
            )


In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=(10, 10),
)

# This function is provided together with the graph
plot_ratp(ax, g)

#### Exercices

- Find the shortest path (Dijkstra) between Félix Faure and Robespierre:

    - with an unweighted graph (all edges and all connections count for 1);
    - with a weighted graph (add the distance information).


- Use the A* algorithm to find the shortest path between Félix Faure and Robespierre.
  Read the documentation to find how to implement your heuristic.

- Is A* algorithm faster than Dijkstra algorithm here?

- Find the fastest path between Félix Faure and Robespierre. Can we use A* here?

  Keep the heuristic for distances and use A* algorithm anyway. Analyse the result.

- How would you compute the fastest path between Félix Faure and Robespierre considering you want to leave "now"?
  The issue here is to take the waiting time into account.

In [ ]:
# Write your code for shortest path here
def shortest_path(g, source, target, method, **kwargs):


In [ ]:
# %load solutions/shortest_path.py


In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

plot_ratp(ax, g, color='lightgrey')

plot_path(
    ax, g, shortest_path(
        g, 'Félix Faure', 'Robespierre',
        nx.shortest_paths.dijkstra_path
    )
)


In [ ]:
# Write your code for A* here


In [ ]:
# %load solutions/ratp_distance.py


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

plot_ratp(ax, g, color='lightgrey')
plot_path(
    ax, g, shortest_path(
        g, 'Félix Faure', 'Robespierre',
        nx.shortest_paths.dijkstra_path,
        weight='distance'
    )
)

In [ ]:
%%timeit
shortest_path(
    g, 'Félix Faure', 'Robespierre',
    nx.shortest_paths.dijkstra_path,
    weight='distance'
)

In [ ]:
%%timeit
shortest_path(
    g, 'Félix Faure', 'Robespierre',
    nx.shortest_paths.astar_path,
    weight='distance',
    heuristic=distance
)

In [ ]:
astar = shortest_path(
    g, 'Félix Faure', 'Robespierre',
    nx.shortest_paths.astar_path,
    weight='distance',
    heuristic=distance,
)

dijkstra = shortest_path(
    g, 'Félix Faure', 'Robespierre',
    # A* without heuristic is equivalent to Dijkstra
    nx.shortest_paths.astar_path,
    weight='distance',
)

# of course the result is the same!
astar['weight'], dijkstra['weight']

In [ ]:
# number of nodes explored
astar['counter'], dijkstra['counter']

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

plot_ratp(ax1, g, color='lightgrey')
plot_ratp(ax2, g, color='lightgrey')

ax1.set_title("Dijkstra algorithm", fontsize=15)
ax2.set_title("A* algorithm", fontsize=15)

def animate():
    dijkstra_animate = animate_path(ax1, dijkstra)
    astar_animate = animate_path(ax2, astar)

    def animate_both(i):
        return dijkstra_animate(i) + astar_animate(i)

    return animate_both

animation.FuncAnimation(
    fig, animate(),
    frames=150, interval=100, blit=True
)

In [ ]:
# %load solutions/ratp_duration.py


In [ ]:
dijkstra = shortest_path(
    g, 'Félix Faure', 'Robespierre',
    nx.shortest_paths.dijkstra_path,
    weight='duration'
)

astar = shortest_path(
    g, 'Félix Faure', 'Robespierre',
    nx.shortest_paths.astar_path,
    weight='duration',
    heuristic=distance
)

# With a wrong heuristic, A* gives nonsense
astar['weight'], dijkstra['weight']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

plot_ratp(ax, g, color='lightgrey')
plot_path(ax, g, dijkstra)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

plot_ratp(ax, g, color='lightgrey')
plot_path(ax, g, astar)